In [ ]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

Saving filtered_data(고가형)(찐최종).csv to filtered_data(고가형)(찐최종).csv
dict_keys(['filtered_data(고가형)(찐최종).csv'])


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/filtered_data(고가형)(찐최종).csv')

In [ ]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,lcom_count,ccom_count,lcafe_count,hcafe_count,centroid,density,density_rank
0,0,0,0,8,0,0,0,0,0,0,POINT (127.071586 37.429794),0,0
1,0,0,0,0,0,0,0,0,1,0,POINT (126.900586 37.432793999999994),0,0
2,0,0,0,4,0,0,0,0,0,0,POINT (126.90358600000002 37.432793999999994),0,0
3,0,0,1,0,0,0,0,0,0,0,POINT (127.12858600000001 37.432793999999994),0,0
4,0,0,0,0,0,0,0,0,0,1,POINT (126.900586 37.435794),1,0


In [ ]:
# 1. 'centroid' 열 삭제
df = df.drop(columns=['centroid'])

# 2. 'rank' 열의 항목과 각 항목의 갯수 확인
rank_counts = df['density_rank'].value_counts()

print(df)  # 업데이트된 DataFrame 출력
print(rank_counts)  # rank 열의 항목들과 갯수 출력

      school_count  academy_count  subway_count  bus_count  apt_count  \
0                0              0             0          8          0   
1                0              0             0          0          0   
2                0              0             0          4          0   
3                0              0             1          0          0   
4                0              0             0          0          0   
...            ...            ...           ...        ...        ...   
4590             0              0             0         20          0   
4591             0              0             0          4          0   
4592             0              0             0          4          0   
4593             1              0             0          0          0   
4594             0              0             1          0          0   

      univ_count  lcom_count  ccom_count  lcafe_count  hcafe_count  density  \
0              0           0           0    

In [ ]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,lcom_count,ccom_count,lcafe_count,hcafe_count,density,density_rank
0,0,0,0,8,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,4,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,1,0


In [ ]:
# 1. 'centroid' 열 삭제
df = df.drop(columns=['density'])

In [ ]:
df['company'] = df['lcom_count'] + df['ccom_count']
df = df.drop(columns=['lcom_count'])
df = df.drop(columns=['ccom_count'])

In [ ]:
df = df.drop(columns=['lcafe_count'])

In [ ]:
df = df.drop(columns=['hcafe_count'])

In [ ]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,density_rank,company
0,0,0,0,8,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,4,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
data = df[['school_count', 'academy_count', 'bus_count','subway_count', 'apt_count', 'univ_count','company']].to_numpy()
target = df['density_rank'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data ,target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# CPU를 모두 이용해 병렬로 교차 검증 수행
# return_train_score=True: 훈련 세트에 대한 점수도 같이 반환합니다. => 과대적합 파악에 유용
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.9973541965122431 0.8905151032797809

0.9916348736026203 0.9844942324755989


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것.
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. => 과대적합 줄일 수o

[0.04711217 0.36684648 0.34339274 0.03018273 0.08420113 0.01811946
 0.11014529]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# oob_score의 기본은 false입니다.
# oob: 부트스트랩 샘플에 포함되지 않고 남은 샘플.
rf.fit(train_input, train_target)
print(rf.oob_score_)
# 0.8934000384837406
# oob로 자체적 평가 => 교차 검증을 대신할 수 있어 훈련 세트에 더 많은 샘플을 이용할 수 있다.

0.9831338411316648


In [ ]:
# 모델 훈련
rf.fit(train_input, train_target)

# 각 트리의 깊이를 리스트로 저장
tree_depths = [estimator.tree_.max_depth for estimator in rf.estimators_]

# 랜덤 포레스트 내 트리들의 최대 깊이 출력
max_tree_depth = max(tree_depths)
print("랜덤 포레스트 내 트리들의 최대 깊이:", max_tree_depth)

# 모든 트리의 깊이 출력 (선택사항)
print("모든 트리들의 깊이:", tree_depths)


랜덤 포레스트 내 트리들의 최대 깊이: 19
모든 트리들의 깊이: [14, 18, 16, 14, 15, 18, 18, 14, 16, 15, 16, 16, 15, 16, 15, 17, 13, 16, 16, 16, 14, 16, 15, 15, 18, 14, 17, 14, 16, 15, 16, 16, 17, 18, 14, 16, 15, 16, 16, 13, 15, 18, 16, 14, 14, 13, 18, 16, 14, 17, 17, 16, 16, 13, 16, 17, 16, 15, 15, 16, 16, 16, 18, 19, 17, 14, 15, 18, 16, 16, 15, 14, 15, 17, 15, 13, 15, 15, 17, 17, 14, 15, 15, 16, 17, 17, 15, 15, 18, 15, 16, 16, 14, 18, 17, 16, 17, 16, 15, 17]


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(2, 5, 1), 'min_samples_split': range(2, 100, 10)}
# arange(): 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다.
# range(): (정수만 사용) 위의 함수와 동일
# 따라서 9 x 10 x 10 = 1350개의 모델을 만든다. 기본 5-폴드 교차를 수행하니 모델의 수는 6750개이다.

gs = GridSearchCV(RandomForestClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
# {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

print(np.max(gs.cv_results_['mean_test_score']))
# 0.8683865773302731

{'max_depth': 2, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}
0.9858544069801833


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier

# 데이터 준비
data = df[['school_count', 'academy_count', 'bus_count', 'subway_count', 'apt_count', 'univ_count', 'company']].to_numpy()
target = df['density_rank'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# RandomForestClassifier 객체 생성 시 하이퍼파라미터 설정
rf = RandomForestClassifier(n_jobs=-1, random_state=42, max_depth=2, min_impurity_decrease=0.0001, min_samples_split=2)

# 교차 검증 수행
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

# 교차 검증 결과 출력
print("훈련 세트 평균 점수:", np.mean(scores['train_score']))
print("테스트 세트 평균 점수:", np.mean(scores['test_score']))


훈련 세트 평균 점수: 0.9858542029528575
테스트 세트 평균 점수: 0.9858544069801833


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것.
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. => 과대적합 줄일 수o

[0.00765164 0.12026188 0.16542006 0.00798897 0.17333439 0.03658479
 0.48875827]


In [ ]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,density_rank,company
0,0,0,0,8,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,4,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0
